In [2]:
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.agents import load_tools, AgentType, initialize_agent
from langchain import LLMMathChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
import os
import openai

In [3]:
inventory_df = pd.read_csv('liquor_amounts.csv')
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [4]:
inventory_df

,liquor,amount_in_liters,cost_per_liter
0,Absinthe,991,77.64
1,Agave Nectar,92,78.70
2,Akvavit,327,13.37
3,Almond Liqueur,876,78.01
4,Amaretto,902,17.09
...,...,...,...
101,Vodka,893,61.57
102,Whiskey,292,94.34
103,White Rum,631,56.93
104,White Wine,352,66.58


In [ ]:
# Convert the 2nd and 3rd columns to floats


KeyError: 'amount_in_liters'

In [37]:
from pydantic import BaseModel, Field, validator
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import create_csv_agent
from langchain.output_parsers import PydanticOutputParser
from typing import List
from langchain.prompts import ChatPromptTemplate, PromptTemplate



# Get the openai api keys
# Get the OpenAI API key and org key from the .env file
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")
os.environ["WOLFRAM_ALPHA_APPID"] = os.getenv("WOLFRAM_APP_ID")
os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID2")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_KEY")

csv_agent = create_csv_agent(llm = llm, path = 'liquor_amounts.csv', verbose=True)



tools = load_tools(['python_repl', 'wolfram-alpha', 'llm-math'], llm=llm)

search = GoogleSearchAPIWrapper()

extra_tools = [
    Tool.from_function(
    func = csv_agent.run, 
    name = 'csv_agent',
    description= "Useful for when you need to query the csv file"
    ),
    Tool.from_function(
    name = "Google Search",
    description = "Search Google for recent results",
    func = search.run
    )]

tools = tools + extra_tools

agent = initialize_agent(tools, llm, agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

class Response(BaseModel):
    amounts: List[str] = Field(description="The liquors and the amounts to query for")
    answer: List[str] = Field(description="A python list of the returned costs of the amount of each liquor")

parser = PydanticOutputParser(pydantic_object=Response)


prompt = PromptTemplate(
    template = "Answer the user's query.\n{format_instructions}\n\nQuestion: How much would the cost of {liquor_amounts} be\n",
    input_variables=['liquor_amounts'],
    partial_variables={'format_instructions':
                       parser.get_format_instructions()}
)

query_amounts = ['3 oz Absinthe', '2 oz Vodka']

_input = prompt.format_prompt(liquor_amounts = query_amounts)

response = csv_agent.run(_input.to_string)

parser.parse(response)




> Entering new AgentExecutor chain...
Thought: We need to extract the cost of the given liquors and amounts from the dataframe and calculate the total cost.
Action: Use pandas dataframe's `loc` method to filter the rows for the given liquors and amounts, then calculate the total cost.
Action Input:
```
liquor_amounts = ['3 oz Absinthe', '2 oz Vodka']
liquor_amounts_df = pd.DataFrame([{'liquor': 'Absinthe', 'amount_in_liters': 0.0887206, 'cost_per_liter': 77.64},
                                  {'liquor': 'Vodka', 'amount_in_liters': 0.0591471, 'cost_per_liter': 13.54}])
liquor_amounts_df['total_cost'] = liquor_amounts_df['amount_in_liters'] * liquor_amounts_df['cost_per_liter']
total_cost = liquor_amounts_df['total_cost'].sum()
answer = [str(total_cost)]
```
Observation: Use pandas dataframe's `loc` method to filter the rows for the given liquors and amounts, then calculate the total cost. is not a valid tool, try another one.
Thought:We need to import pandas first before we can us

In [ ]:
import random
# Add a column with the name of "cost_per_liter" and populate it with random floats between 10 and 100
inventory_df['cost_per_liter'] = inventory_df['amount_in_liters'].apply(lambda x: round(random.uniform(10, 100), 2))

In [33]:
# Change the name of the "amount_in_liters" column to "inventory_amount"
inventory_df = inventory_df.rename(columns={'amount_in_liters': 'inventory_amount'})

In [7]:
inventory_df.to_csv('liquor_amounts.csv', index=False)

In [52]:
# Convert the "amount_in_liters" and "cost_per_liter" columns to floats
inventory_df['inventory_amount'] = inventory_df['inventory_amount'].astype(float)
inventory_df['cost_per_liter'] = inventory_df['cost_per_liter'].astype(float)

In [53]:
from langchain.utilities import WolframAlphaAPIWrapper, PythonREPL
from langchain.agents import AgentType, create_csv_agent, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.chains import SimpleSequentialChain
from langchain.output_parsers import PydanticOutputParser


wolfram = WolframAlphaAPIWrapper(wolfram_alpha_appid=WOLFRAM_ALPHA_APPID)
python_repl = PythonREPL()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(tool_names = ["wolfram-alpha"], verbose = True, wolfram_alpha_appid = WOLFRAM_ALPHA_APPID, llm=llm)

# Use the PydancticOutputParser to parse the output of the agent call as a Pydantic model
csv_agent = create_csv_agent(llm = llm, path = 'liquor_amounts.csv', verbose = True, tools = tools)



tools = [Tool(
    name="csv_agent",
    func = csv_agent.run,
    description="Useful when you need to query data from a CSV file.",
    ),
    Tool(
    name="wolfram-alpha",
    func = wolfram.run,
    description="Useful when you need to query data from Wolfram Alpha, run calculations, or get information about a specific topic.",
    ),
    Tool(
    name="python-repl",
    func = python_repl.run,
    description="Useful when you need to run Python code.",
    )
]
agent = initialize_agent(tools = tools, llm = llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

response = agent.run("What is the cost per ounce of Chambord in the liquor_amounts.csv file?")
response



> Entering new AgentExecutor chain...
I need to query data from a CSV file.
Action: csv_agent
Action Input: "What is the cost per ounce of Chambord in liquor_amounts.csv?"

> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Thought: We need to filter the dataframe to only include Chambord, then calculate the cost per ounce.
Action: 
```python_repl_ast
chambord_df = df[df['liquor'] == 'Chambord']
cost_per_ounce = chambord_df['cost_per_liter'] / 33.814
cost_per_ounce
````

In [4]:
from langchain.chains import LLMChain
chain = LLMChain(llm = chat, prompt = chat_prompt_template)


In [6]:
spirits_list = [
    'Absinthe',
    'Agave Nectar',
    'Akvavit',
    'Almond Liqueur',
    'Amaretto',
    'Aperol',
    'Apple Brandy',
    'Applejack',
    'Apricot Brandy',
    'Averna',
    'Bailey\'s Irish Cream',
    'Bénédictine',
    'Bitters',
    'Black Sambuca',
    'Blue Curaçao',
    'Bourbon',
    'Brandy',
    'Campari',
    'Chambord',
    'Cherry Brandy',
    'Cider',
    'Cinnamon Schnapps',
    'Cocchi Americano',
    'Cognac',
    'Cointreau',
    'Crème de Cacao',
    'Crème de Cassis',
    'Crème de Menthe',
    'Crème de Violette',
    'Curacao',
    'Drambuie',
    'Dry Vermouth',
    'Elderflower Liqueur',
    'Falernum',
    'Fernet Branca',
    'Galliano',
    'Genever',
    'Gin',
    'Gold Rum',
    'Goldschläger',
    'Grand Marnier',
    'Grapefruit Liqueur',
    'Green Chartreuse',
    'Grenadine',
    'Hazelnut Liqueur',
    'Hpnotiq',
    'Irish Cream',
    'Jägermeister',
    'Kahlua',
    'Lemon Juice',
    'Lemoncello',
    'Lillet',
    'Lime Juice',
    'Maraschino Liqueur',
    'Mezcal',
    'Midori',
    'Mint Leaves',
    'Nutmeg',
    'Orange Bitters',
    'Orange Curacao',
    'Orange Flower Water',
    'Orange Juice',
    'Orgeat Syrup',
    'Overproof Rum',
    'Peach Bitters',
    'Peach Liqueur',
    'Peppermint Schnapps',
    'Pernod',
    'Peychaud\'s Bitters',
    'Pimm\'s No. 1',
    'Pineapple Juice',
    'Port',
    'Prosecco',
    'Pumpkin Liqueur',
    'Raspberry Liqueur',
    'Red Wine',
    'Rhum Agricole',
    'Rose Water',
    'Rum',
    'Rye Whiskey',
    'Sake',
    'Sambuca',
    'Scotch',
    'Sherry',
    'Simple Syrup',
    'Sloe Gin',
    'Smoked Paprika',
    'Southern Comfort',
    'Sparkling Wine',
    'Spiced Rum',
    'St. Germain',
    'Strawberry Liqueur',
    'Sweet and Sour Mix',
    'Sweet Vermouth',
    'Tabasco Sauce',
    'Tequila',
    'Tomato Juice',
    'Triple Sec',
    'Vanilla Liqueur',
    'Vanilla Syrup',
    'Vermouth',
    'Vodka',
    'Whiskey',
    'White Rum',
    'White Wine',
    'Yellow Chartreuse'
]



In [7]:
# Create a pandas dataframe with the list of spirits
df = pd.DataFrame(spirits_list, columns = ["liquor"])

In [8]:
import numpy as np
# Add a column to the dataframe called "amount" and generate a random number between 1 and 1000 for each row
df["amount"] = np.random.randint(1, 1000, df.shape[0])


In [9]:
# Change the name of the "amount" columns to "amount_in_oz"
df.rename(columns = {"amount": "amount_in_oz"}, inplace = True)
df

,liquor,amount_in_oz
0,Absinthe,991
1,Agave Nectar,92
2,Akvavit,327
3,Almond Liqueur,876
4,Amaretto,902
...,...,...
101,Vodka,893
102,Whiskey,292
103,White Rum,631
104,White Wine,352


In [10]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", max_tokens = 750, temperature = 0.7, top_p = 0.9, frequency_penalty = 0.5, presence_penalty = 0.5,\
            openai_api_key = openai.api_key, openai_organization =  openai.organization)


In [13]:
# Export the dataframe to a csv file
df.to_csv("liquor_amounts.csv", index = False)

from langchain.agents import create_csv_agent

In [14]:
# Create the agent using the csv file
agent = create_csv_agent(llm, "liquor_amounts.csv", verbose = True)


In [15]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [18]:
class SpiritsList(BaseModel):
    spirits: List[str] = Field(description = "A list of spirits")
    amounts: List[int] = Field(description = "A list of amounts in oz")

    
spirits_query = "Return a list of spirits in the .csv file and their amounts in ml.  You will need to convert\
                the amounts in oz to ml."

parser = PydanticOutputParser(pydantic_object=SpiritsList, query=spirits_query)

spirits_and_amounts = agent.run(input = "Return a list of spirits and their amounts in ml", parser=parser)


     



> Entering new AgentExecutor chain...
Thought: We need to select the 'liquor' and 'amount_in_oz' columns, convert the amount to ml, and return the result as a list.
Action: python_repl_ast
Action Input:
```
df['amount_in_ml'] = df['amount_in_oz'] * 29.5735
result = df[['liquor', 'amount_in_ml']].values.tolist()
```
Observation: 
Thought:We have created a new column 'amount_in_ml' by multiplying the 'amount_in_oz' column by 29.5735 to convert it to ml. We then selected the 'liquor' and 'amount_in_ml' columns using double brackets and converted the result to a list using the .values.tolist() method.
Action: python_repl_ast
Action Input:
```
result
```
Observation: [['Absinthe', 29307.338499999998], ['Agave Nectar', 2720.7619999999997], ['Akvavit', 9670.5345], ['Almond Liqueur', 25906.386], ['Amaretto', 26675.297], ['Aperol', 14757.1765], ['Apple Brandy', 22327.9925], ['Applejack', 8428.4475], ['Apricot Brandy', 3371.379], ['Averna', 13071.487], ["Bailey's Irish Cream", 20524.009], ['Bé

In [15]:
spirits_and_amounts

'[(\'Absinthe\', 537), (\'Agave Nectar\', 941), (\'Akvavit\', 940), (\'Almond Liqueur\', 653), (\'Amaretto\', 741), (\'Aperol\', 751), (\'Apple Brandy\', 872), (\'Applejack\', 903), (\'Apricot Brandy\', 164), (\'Averna\', 372), ("Bailey\'s Irish Cream", 913...)]'

In [19]:
print(chain.run(liquor = "rum", cocktail_type = "craft", cuisine = "cuban", theme = "summer", spirits_and_amounts = spirits_and_amounts))

Recipe Name: Havana Sunset

Ingredients:
- 1 oz rum
- 1 oz apricot brandy
- 1/2 oz amaretto
- 1/2 oz agave nectar
- 1/4 oz absinthe
- 1 oz orange juice
- 1/2 oz lime juice
- Splash of grenadine

Instructions:
1. In a cocktail shaker filled with ice, combine the rum, apricot brandy, amaretto, agave nectar, and absinthe.
2. Shake well and strain into a tall glass filled with ice.
3. Add the orange juice and lime juice to the glass and stir.
4. Slowly pour in a splash of grenadine to create a sunset effect.
5. Garnish with a slice of orange or lime.

Enjoy your tropical Havana Sunset cocktail that pairs perfectly with Cuban cuisine!
